
#RAG-BSc

using chroma & gradio

PIP's

In [1]:
%pip install sentence-transformers transformers==4.32.0 tf-keras langchain_community langchain_text_splitters chromadb torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 3.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of sentence-transformers to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of sentence-transformers to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 14.

In [2]:
%pip install --upgrade transformers sentence-transformers
%pip install pypdf numpy

  Using cached sentence_transformers-4.0.2-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 96.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.32.0
    Uninstalling transformers-4.32.0:
      Successfully uninstalled transformers-4.32.0
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.6.1
    Uninstalling sentence-transformers-2.6.1:
      Successfully uninstalled sentence-transformers-2.6.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 6.1 MB/s eta 0:00:00


In [3]:
%pip install chromadb
%pip install gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 119.2 MB/s eta 0:00:00


In [ ]:
!pip list

Package                                  Version
---------------------------------------- -------------------
absl-py                                  1.4.0
accelerate                               1.5.2
aiohappyeyeballs                         2.6.1
aiohttp                                  3.11.15
aiosignal                                1.3.2
alabaster                                1.0.0
albucore                                 0.0.23
albumentations                           2.0.5
ale-py                                   0.10.2
altair                                   5.5.0
annotated-types                          0.7.0
anyio                                    4.9.0
argon2-cffi                              23.1.0
argon2-cffi-bindings                     21.2.0
array_record                             0.7.1
arviz                                    0.21.0
asgiref                                  3.8.1
astropy                                  7.0.1
astropy-iers-data                    

Import the necessary libraries

In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from chromadb.config import Settings

Load and cut a PDF document into pieces of text

In [5]:
loader = PyPDFLoader("BSc_Student Handbook_EN_M.pdf")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = text_splitter.split_documents(docs)
text_lines = [chunk.page_content for chunk in chunks]

In [6]:
def encoding_text(text_lines):
    embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

    if not text_lines:
        raise ValueError("La liste de textes est vide.")

    embeddings = embedding_model.encode(text_lines)
    return embeddings

Create chroma_db

In [7]:
import chromadb
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection(name="rag_collection")


In [8]:
for i, line in enumerate(text_lines):
    collection.add(
        ids=[str(i)],
        embeddings=[encoding_text([line])[0].tolist()],  # Convertir en liste
        metadatas=[{"text": line}]
    )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Demonstration of the different responses of the models

In [27]:
import gradio as gr
from transformers import pipeline
import chromadb
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

client = chromadb.PersistentClient(path="chroma_db")
collection = client.get_or_create_collection(name="rag_collection")

MODELS = {
    "RoBERTa": "deepset/roberta-base-squad2",
    "ELECTRA": "google/electra-small-discriminator",
    "DeBERTa": "deepset/deberta-v3-base-squad2"
}

def get_answers(question):
    query_embedding = embedding_model.encode(question).tolist()
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=2
    )

    if results.get("metadatas") and results["metadatas"][0]:
        context = "\n".join([hit["text"] for sublist in results["metadatas"] for hit in sublist if "text" in hit])
    else:
        context = " ".join(results["documents"][0])

    answers = {}
    for model_name, model_path in MODELS.items():
        try:
            qa_model = pipeline("question-answering", model=model_path)
            answer = qa_model(question=question, context=context)["answer"]
            answers[model_name] = answer
        except Exception as e:
            answers[model_name] = f"Error: {str(e)}"

    output = ""
    for name, answer in answers.items():
        output += f"{name}: {answer}\n\n"

    return output.strip()

demo = gr.Interface(
    fn=get_answers,
    inputs=gr.Textbox(label="Your question"),
    outputs=gr.Textbox(label="Model responses", lines=10),
    title="🤖 QA Model Comparator"
)

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6f609c8ac0d2447fc0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Demo with the best model

In [13]:
# Initialize the embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Initialize Q&A pipeline
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

# Function to encode text
def encoding_text(text):
    return model.encode(text)

# Load the existing DB Chroma
chroma_client = chromadb.PersistentClient(path="chroma_db")
collection = client.get_or_create_collection(name="rag_collection")

def ask_question(question):
    # Encode the question
    embedding = encoding_text([question])[0].tolist()

    # Search for the nearest documents
    search_res = collection.query(
        query_embeddings=[embedding],
        n_results=3
    )

    # Extract metadata context
    results = search_res["metadatas"]
    context = "\n".join([hit["text"] for sublist in results for hit in sublist if "text" in hit])

    # Creating the prompt
    PROMPT = """
    Use the information enclosed in <context> tags to provide an answer to the
    question enclosed in <question> tags.
    <context>
    {context}
    </context>
    <question>
    {question}
    </question>
    Réponse:
    """
    prompt = PROMPT.format(context=context, question=question)

    # Generate a response with the QA model
    response = qa_pipeline(question=question, context=context)

    return response["answer"]

# Demo Gradio
iface = gr.Interface(
    fn=ask_question,
    inputs=gr.Textbox(label="Pose ta question ici"),
    outputs=gr.Textbox(label="Réponse RAG (basée sur le Handbook)"),
    title="📚 RAG Demo - BSc Student Handbook"
)

iface.launch()

Device set to use cuda:0


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1955bf1e98947d884c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
